Given the string “US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20”

where each element follows, sourceCountry:targetCountry:method:cost

### Part 1

Write a function shipping_cost(inputString, sourceCountry, targetCountry, method) to return the cost

*pay attention to the edge cases*

In [2]:
## Part 1
## use string match
def shipping_cost(inputString, sourceCountry, targetCountry, method):

    # parse the input string
    for i in inputString.split(","):
        if len(i.split(":")) == 4:
            source, target, method, cost = i.split(":")
            if source == sourceCountry and target == targetCountry and method == method:
                return int(cost)

    return None # if no match

In [3]:
input_data = "US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20"
print(shipping_cost(input_data, "US", "UK", "UPS"))  # Output: 4
print(shipping_cost(input_data, "US", "UK", "DHL"))  # Output: 5
print(shipping_cost(input_data, "UK", "CA", "FedEx"))  # Output: 10
print(shipping_cost(input_data, "AU", "JP", "DHL"))  # Output: 20
print(shipping_cost(input_data, "US", "CA", "FedEx"))  # Output: None (No match)

4
4
10
20
None


In [4]:
### part 1
### use nested map
def build_shipping_dict(inputString):
    shipping_dict = {}
    for record in inputString.split(','):
        if len(record.split(":")) == 4:
            source, target, method, cost = record.split(":")
            cost = int(cost)

            # the following 2 if clause is to avoid KeyError
            if source not in shipping_dict:
                shipping_dict[source] = {}
            if target not in shipping_dict[source]:
                shipping_dict[source][target] = {}
            
            shipping_dict[source][target][method] = cost
        
    return shipping_dict


In [5]:
def shipping_cost(inputString, sourceCountry, targetCountry, method):
    
    shipping_dict = build_shipping_dict(inputString)
    
    # use get function to avoid no match cases
    # dictionary.get(keyname, value): a value to return if the specified key does not exist
    # for the first 2 get function, specifying {} is to avoid the error that default None does not have a get function
    return shipping_dict.get(sourceCountry, {}).get(targetCountry, {}).get(method, None)

In [6]:
input_data = "US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20"
print(shipping_cost(input_data, "US", "UK", "UPS"))  # Output: 4
print(shipping_cost(input_data, "US", "UK", "DHL"))  # Output: 5
print(shipping_cost(input_data, "UK", "CA", "FedEx"))  # Output: 10
print(shipping_cost(input_data, "AU", "JP", "DHL"))  # Output: 20
print(shipping_cost(input_data, "US", "CA", "FedEx"))  # Output: None (No match)
print(shipping_cost(input_data, "CN", "CA", "FedEx"))  # Output: None (No match)

4
5
10
20
None
None


### Part 2
If one middle country is allowed, output the following structure for the example input sourceCountry=US, targetCountry=CA, 

{
    route: "US -> UK -> CA", 
    method: "UPS -> FedEx", 
    cost: 14
}

In [13]:
from collections import defaultdict

### part 2
def shipping_cost(inputString, sourceCountry, targetCountry):
    shipping_dict = build_shipping_dict(inputString)

    result_route = ""
    result_method = ""
    result_cost = ""
    result = []

    if sourceCountry in shipping_dict:
        # all possible one stop location from source country
        one_stop_loc = list(shipping_dict[sourceCountry].keys())

        for loc in one_stop_loc:
            if loc == targetCountry:
                for method, cost in shipping_dict[sourceCountry][targetCountry].items():
                    result_route = sourceCountry + " -> " + targetCountry
                    result_method = method
                    result_cost = cost
                    result.append({"route": result_route, "method": result_method, "cost": result_cost})
            if loc in shipping_dict and targetCountry in shipping_dict[loc]:
                # construct the final output                
                # there could be multiple routes between sourceCountry to loc and loc to TargetCountry
                for first_method, first_cost in shipping_dict[sourceCountry][loc].items():
                    for second_method, second_cost in shipping_dict[loc][targetCountry].items():
                        result_route = sourceCountry + " -> " + loc + " -> " + targetCountry
                        result_method = first_method + " -> " + second_method
                        result_cost = first_cost + second_cost
                        result.append({"route": result_route, "method": result_method, "cost": result_cost})
        
    if len(result) == 0:
        return None
    else:
        return result


In [10]:
shipping_dict = build_shipping_dict("US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20,US:ME:UPS:1,ME:CA:FedEx:2")
print(shipping_dict.get("CA", {}))

{}


In [12]:
input_data = "US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20"
print(shipping_cost(input_data, "CN", "CA"))

None


### Part 3
Find the lowest cost

In [14]:
from collections import defaultdict

### part 3
def shipping_cost(inputString, sourceCountry, targetCountry):
    shipping_dict = build_shipping_dict(inputString)

    result_route = ""
    result_method = ""
    result_cost = ""
    min_cost = float("inf")

    if sourceCountry in shipping_dict:
        # all possible one stop location from source country
        one_stop_loc = list(shipping_dict[sourceCountry].keys())

        for loc in one_stop_loc:
            if loc == targetCountry:
                for method, cost in shipping_dict[sourceCountry][targetCountry].items():
                    result_route = sourceCountry + " -> " + targetCountry
                    result_method = method
                    result_cost = cost

                    if result_cost < min_cost:
                        result = {"route": result_route, "method": result_method, "cost": result_cost}
                        min_cost = result_cost
            if loc in shipping_dict and targetCountry in shipping_dict[loc]:
                # construct the final output                
                # there could be multiple routes between sourceCountry to loc and loc to TargetCountry
                for first_method, first_cost in shipping_dict[sourceCountry][loc].items():
                    for second_method, second_cost in shipping_dict[loc][targetCountry].items():
                        result_route = sourceCountry + " -> " + loc + " -> " + targetCountry
                        result_method = first_method + " -> " + second_method
                        result_cost = first_cost + second_cost
                        if result_cost < min_cost:
                            result = {"route": result_route, "method": result_method, "cost": result_cost}
                            min_cost = result_cost
        
    if len(result) == 0:
        return None
    else:
        return result


In [15]:
input_data = "US:UK:UPS:4,US:UK:DHL:5,UK:CA:FedEx:10,AU:JP:DHL:20,US:ME:UPS:1,ME:CA:FedEx:2"
print(shipping_cost(input_data, "US", "CA"))

{'route': 'US -> ME -> CA', 'method': 'UPS -> FedEx', 'cost': 3}
